In [1]:
!nvidia-smi

Tue May  4 03:41:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%load_ext tensorboard
%load_ext autoreload
%autoreload 2

In [11]:
%run ../nli/config.py
%run ../nli/nli_finetune.py
%run ../nli/mnli.py
%run ../nli/nli_dataset.py
%run ../augmentations/transforms.py

In [4]:
import pandas as pd

import torch
import transformers
import torchmetrics
import pytorch_lightning as pl

In [5]:
_=pl.seed_everything(CONFIG['SEED'])

Global seed set to 13


## Setup

In [6]:
CONFIG["MODEL_NAME_OR_PATH"]="emilyalsentzer/Bio_ClinicalBERT"
CONFIG["MODEL_SAVE_NAME"]="BIO_Clinical_BERT_PERTUBATIONS"
CONFIG["MAX_EPOCHS"]=10

In [12]:
%%capture
# Commment this cell to avoid excessive RAM usage when not training with perturbations
train_transforms=ClinicalSynonymSubstitution(substitution_probability=0.9,p=0.9)
val_transforms=ClinicalSynonymSubstitution(substitution_probability=0.9,p=0.9)

## Training

In [13]:
model_save_checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    dirpath=CONFIG['CHECKPOINT_DIR'],
    filename=f"{CONFIG['MODEL_SAVE_NAME']}"+'-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min',
)

/home/va2134/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Checkpoint directory ./checkpoints exists and is not empty.
  warnings.warn(*args, **kwargs)


In [14]:
trainer = pl.Trainer(gpus=torch.cuda.device_count(),
                     max_epochs=CONFIG['MAX_EPOCHS'],
                     callbacks=[model_save_checkpoint],
                     precision=CONFIG['PRECISION'],
                     num_sanity_val_steps=0
                    )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.


In [16]:
model=NLIFineTuningModel(
    model_name_or_path=CONFIG['MODEL_NAME_OR_PATH'],
    num_labels=CONFIG['NUM_CLASSES'],
    learning_rate=CONFIG['LEARNING_RATE'],
    adam_epsilon=CONFIG['ADAM_EPSILON'],
    weight_decay=CONFIG['WEIGHT_DECAY'],
    max_len=CONFIG['MAX_LEN'],
    warmup_steps=CONFIG['WARMUP_STEPS'],
    max_epochs=trainer.max_epochs,
    gpus=trainer.gpus,
    accumulate_grad_batches=trainer.accumulate_grad_batches,
)

mnli_dm=NLIDataModule(get_split_def=mnli_df,
                     train_transforms=train_transforms, # Comment these 2 args when not training with pertubations
                     val_transforms=val_transforms
                     )
trainer.fit(model,mnli_dm)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

Training: 0it [00:00, ?it/s]

/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]
/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray fro

Validating: 0it [00:00, ?it/s]

/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray fro

...

/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray fro

Validating: 0it [00:00, ?it/s]

/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray fro

Validating: 0it [00:00, ?it/s]

/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]
/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray fro

In [17]:
trainer.logged_metrics

{'train_accuracy': tensor(0.9222, device='cuda:0'),
 'train_f1': tensor(0.9223, device='cuda:0'),
 'epoch': tensor(9.),
 'train_loss': tensor(0.2246, device='cuda:0'),
 'val_accuracy': tensor(0.8057),
 'val_f1': tensor(0.8060),
 'val_loss': tensor(0.6271)}